In [1]:
showYOLO = False

In [2]:

%cd ~/cloudfiles/code/Users/dnn200001/yolov5
# install dependencies as necessary
#!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-inference/code/Users/dnn200001/yolov5
Setup complete. Using torch 1.12.0 _CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)


/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%cd ~/cloudfiles/code/Users/dnn200001

import os
import urllib.error
import urllib.request

import csv
import cv2
import itertools
import numpy as np
import pandas as pd
import os
import shutil
import sys
import tempfile
import tqdm

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-inference/code/Users/dnn200001


2023-04-27 18:30:00.890724: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 18:30:01.971585: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/cv2/../../lib64:
2023-04-27 18:30:01.971682: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /anaconda/envs/azureml_py38_PT_TF/lib/python3.8/sit

In [4]:
pose_sample_rpi_path = os.path.join(os.getcwd(), 'examples/lite/examples/pose_estimation/raspberry_pi')
sys.path.append(pose_sample_rpi_path)

# Load MoveNet Thunder model
import utils
from data import BodyPart
from ml import Movenet
movenet = Movenet('movenet_thunder')

# Define function to run pose estimation using MoveNet Thunder.
# You'll apply MoveNet's cropping algorithm and run inference multiple times on
# the input image to improve pose estimation accuracy.
def detect(input_tensor, inference_count=3):
  """Runs detection on an input image.
 
  Args:
    input_tensor: A [height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.
    inference_count: Number of times the model should run repeatly on the
      same input image to improve detection accuracy.
 
  Returns:
    A Person entity detected by the MoveNet.SinglePose.
  """
  image_height, image_width, channel = input_tensor.shape
 
  # Detect pose using the full input image
  movenet.detect(input_tensor.numpy(), reset_crop_region=True)
 
  # Repeatedly using previous detection result to identify the region of
  # interest and only croping that region to improve detection accuracy
  for _ in range(inference_count - 1):
    person = movenet.detect(input_tensor.numpy(), 
                            reset_crop_region=False)

  return person

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
#@title Functions to visualize the pose estimation results.

def draw_prediction_on_image(
    image, person, crop_region=None, close_figure=True,
    keep_input_size=False):
  """Draws the keypoint predictions on image.
 
  Args:
    image: An numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    person: A person entity returned from the MoveNet.SinglePose model.
    close_figure: Whether to close the plt figure after the function returns.
    keep_input_size: Whether to keep the size of the input image.
 
  Returns:
    An numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  # Draw the detection result on top of the image.
  image_np = utils.visualize(image, [person])
  
  # Plot the image with detection results.
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  im = ax.imshow(image_np)
 
  if close_figure:
    plt.close(fig)
 
  if not keep_input_size:
    image_np = utils.keep_aspect_ratio_resizer(image_np, (512, 512))

  return image_np

In [6]:
#@title Code to load the images, detect pose landmarks and save them into a CSV file

class MoveNetPreprocessor(object):
  """Helper class to preprocess pose sample images for classification."""
 
  def __init__(self,
               images_in_folder,
               images_out_folder,
               csvs_out_path):
    """Creates a preprocessor to detection pose from images and save as CSV.

    Args:
      images_in_folder: Path to the folder with the input images. It should
        follow this structure:
        yoga_poses
        |__ downdog
            |______ 00000128.jpg
            |______ 00000181.bmp
            |______ ...
        |__ goddess
            |______ 00000243.jpg
            |______ 00000306.jpg
            |______ ...
        ...
      images_out_folder: Path to write the images overlay with detected
        landmarks. These images are useful when you need to debug accuracy
        issues.
      csvs_out_path: Path to write the CSV containing the detected landmark
        coordinates and label of each image that can be used to train a pose
        classification model.
    """
    self._images_in_folder = images_in_folder
    self._images_out_folder = images_out_folder
    self._csvs_out_path = csvs_out_path
    self._messages = []

    # Create a temp dir to store the pose CSVs per class
    self._csvs_out_folder_per_class = tempfile.mkdtemp()
 
    # Get list of pose classes and print image statistics
    self._pose_class_names = sorted(
        [n for n in os.listdir(self._images_in_folder) if not n.startswith('.')]
        )
    
  def process(self, per_pose_class_limit=None, detection_threshold=0.1):
    """Preprocesses images in the given folder.
    Args:
      per_pose_class_limit: Number of images to load. As preprocessing usually
        takes time, this parameter can be specified to make the reduce of the
        dataset for testing.
      detection_threshold: Only keep images with all landmark confidence score
        above this threshold.
    """
    # Loop through the classes and preprocess its images
    for pose_class_name in self._pose_class_names:
      print('Preprocessing', pose_class_name, file=sys.stderr)

      # Paths for the pose class.
      images_in_folder = os.path.join(self._images_in_folder, pose_class_name)
      images_out_folder = os.path.join(self._images_out_folder, pose_class_name)
      csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                  pose_class_name + '.csv')
      if not os.path.exists(images_out_folder):
        os.makedirs(images_out_folder)
 
      # Detect landmarks in each image and write it to a CSV file
      with open(csv_out_path, 'w') as csv_out_file:
        csv_out_writer = csv.writer(csv_out_file, 
                                    delimiter=',', 
                                    quoting=csv.QUOTE_MINIMAL)
        # Get list of images
        image_names = sorted(
            [n for n in os.listdir(images_in_folder) if not n.startswith('.')])
        if per_pose_class_limit is not None:
          image_names = image_names[:per_pose_class_limit]

        valid_image_count = 0
 
        # Detect pose landmarks from each image
        for image_name in tqdm.tqdm(image_names):
          image_path = os.path.join(images_in_folder, image_name)

          try:
            image = tf.io.read_file(image_path)
            image = tf.io.decode_jpeg(image)
          except:
            self._messages.append('Skipped ' + image_path + '. Invalid image.')
            continue
          else:
            image = tf.io.read_file(image_path)
            image = tf.io.decode_jpeg(image)
            image_height, image_width, channel = image.shape
          
          # Skip images that isn't RGB because Movenet requires RGB images
          if channel != 3:
            self._messages.append('Skipped ' + image_path +
                                  '. Image isn\'t in RGB format.')
            continue
          person = detect(image)
          
          # Save landmarks if all landmarks were detected
          min_landmark_score = min(
              [keypoint.score for keypoint in person.keypoints])
          should_keep_image = min_landmark_score >= detection_threshold
          if not should_keep_image:

            self._messages.append('Skipped ' + image_path +
                                  '. No pose was confidentlly detected.')
            continue

          valid_image_count += 1

          # # Draw the prediction result on top of the image for debugging later
          
          # output_overlay = draw_prediction_on_image(
          #     image.numpy().astype(np.uint8), person, 
          #     close_figure=True, keep_input_size=True)
        
          # # Write detection result into an image file
          # output_frame = cv2.cvtColor(output_overlay, cv2.COLOR_RGB2BGR)
          # cv2.imwrite(os.path.join(images_out_folder, image_name), output_frame)
        
          # Get landmarks and scale it to the same size as the input image
          pose_landmarks = np.array(
              [[keypoint.coordinate.x, keypoint.coordinate.y, keypoint.score]
                for keypoint in person.keypoints],
              dtype=np.float32)

          # Write the landmark coordinates to its per-class CSV file
          coordinates = pose_landmarks.flatten().astype(np.str).tolist()
          csv_out_writer.writerow([image_name] + coordinates)

        if not valid_image_count:
          raise RuntimeError(
              'No valid images found for the "{}" class.'
              .format(pose_class_name))
        
      
    # Print the error message collected during preprocessing.
    print('\n'.join(self._messages))

    # Combine all per-class CSVs into a single output file
    all_landmarks_df = self._all_landmarks_as_dataframe()
    all_landmarks_df.to_csv(self._csvs_out_path, index=False)

  def class_names(self):
    """List of classes found in the training dataset."""
    return self._pose_class_names
  
  def _all_landmarks_as_dataframe(self):
    """Merge all per-class CSVs into a single dataframe."""
    total_df = None
    for class_index, class_name in enumerate(self._pose_class_names):
      csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                  class_name + '.csv')
      per_class_df = pd.read_csv(csv_out_path, header=None)
      
      # Add the labels
      per_class_df['class_no'] = [class_index]*len(per_class_df)
      per_class_df['class_name'] = [class_name]*len(per_class_df)

      # Append the folder name to the filename column (first column)
      per_class_df[per_class_df.columns[0]] = (os.path.join(class_name, '') 
        + per_class_df[per_class_df.columns[0]].astype(str))

      if total_df is None:
        # For the first class, assign its data to the total dataframe
        total_df = per_class_df
      else:
        # Concatenate each class's data into the total dataframe
        total_df = pd.concat([total_df, per_class_df], axis=0)
 
    list_name = [[bodypart.name + '_x', bodypart.name + '_y', 
                  bodypart.name + '_score'] for bodypart in BodyPart] 
    header_name = []
    for columns_name in list_name:
      header_name += columns_name
    header_name = ['file_name'] + header_name
    header_map = {total_df.columns[i]: header_name[i] 
                  for i in range(len(header_name))}
 
    total_df.rename(header_map, axis=1, inplace=True)

    return total_df

In [7]:
#download video based on the number of frames
def extract_frames(video_url, frame_rate):
    # Set the directory to store the frames
    output_dir = 'output_frames'

    # Create the directory if it does not exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    else:
        shutil.rmtree(output_dir)
        os.makedirs(output_dir)

    try:
        # Download the video with a fixed filename
        filename = os.path.join(output_dir, 'my_video.mp4')
        urllib.request.urlretrieve(video_url, filename)

        # Open the video file
        cap = cv2.VideoCapture(filename)

        # Get the frame rate of the video
        fps = cap.get(cv2.CAP_PROP_FPS)

        # Calculate the number of frames to skip
        frame_skip = int(fps / frame_rate)

        # Initialize the frame counter
        frame_count = 0

        # Set the directory to store the frames
        video_name = os.path.splitext(os.path.basename(filename))[0]
        frames_dir = os.path.join(output_dir, f"{video_name}_frames")
        if not os.path.exists(frames_dir):
            os.makedirs(frames_dir)

        # Read the frames from the video and save them to the output directory
        while cap.isOpened():
            ret, frame = cap.read()
            if ret:
                if frame_count % frame_skip == 0:
                    frame_path = os.path.join(frames_dir, f'frame_{frame_count:05d}.jpg')
                    cv2.imwrite(frame_path, frame)
                frame_count += 1
            else:
                break

        # Release the video capture object and delete the video file
        cap.release()
        os.remove(filename)

        return frames_dir

    except (urllib.error.HTTPError, urllib.error.URLError, cv2.error) as e:
        print(f"An error occurred while processing the video: {e}")
        return None


In [8]:
def load_pose_landmarks(csv_path):
  """Loads a CSV created by MoveNetPreprocessor.
  
  Returns:
    X: Detected landmark coordinates and scores of shape (N, 17 * 3)
    y: Ground truth labels of shape (N, label_count)
    classes: The list of all class names found in the dataset
    dataframe: The CSV loaded as a Pandas dataframe features (X) and ground
      truth labels (y) to use later to train a pose classification model.
  """

  # Load the CSV file
  dataframe = pd.read_csv(csv_path)
  df_to_process = dataframe.copy()

  # Drop the file_name columns as you don't need it during training.
  df_to_process.drop(columns=['file_name'], inplace=True)

  # Extract the list of class names
  classes = df_to_process.pop('class_name')
  # Extract the labels
  y = df_to_process.pop('class_no')

  # Convert the input features and labels into the correct format for training.
  X = df_to_process.astype('float64')
  y = keras.utils.to_categorical(y)

  return X, y, classes, dataframe

In [9]:
def poseInference(interpreter, X):
  """Evaluates the given TFLite model and return its accuracy."""
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on all given poses.
  y_pred = []
  for i in range(len(X)):
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = X[i: i + 1].astype('float32')
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the class with highest
    # probability.
    output = interpreter.tensor(output_index)
    predicted_label = np.argmax(output()[0])
    y_pred.append(predicted_label)

  # Compare prediction results with ground truth labels to calculate accuracy.
  return y_pred

# Evaluate the accuracy of the converted TFLite model


# Run predictions on the TFLite model



#to do inference
#X_test, y_test, _, df_test = load_pose_landmarks(csvs_out_test_path)
###doInference(classifier_interpreter, X_test))

In [10]:
%cd ~/cloudfiles/code/Users/dnn200001/

DETECT_ROOT = "Users/dnn200001/yolov5/runs/detect"


torch.cuda.empty_cache()
tf.keras.backend.clear_session()

fileName = "MISS"

df = pd.read_csv("VideoLinks/" + fileName + ".csv")
max = 2
count = 0
for index, row in df.iterrows():
    count = count + 1
    if count > max: break
    url = row['link']
    
    print(url)
    source = extract_frames(url, 4)
    !python yolov5/detect.py --weights yolov5/runs/train/yolov5s_results/weights/last.pt --img 480 --conf 0.5 --source {source} --classes 3 --name MISS --save-crop --nosave


/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-inference/code/Users/dnn200001
https://videos.nba.com/nba/pbp/media/2014/10/29/0021400014/4/063bd16c-b31f-a552-d604-80db24080594_1280x720.mp4
detect: weights=['yolov5/runs/train/yolov5s_results/weights/last.pt'], source=output_frames/my_video_frames, data=yolov5/data/coco128.yaml, imgsz=[480, 480], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=True, nosave=True, classes=[3], agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=MISS, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.10 torch-1.8.1+cu102 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 7268094 parameters, 0 gradients, 16.8 GFLOPs
image 1/47 /mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-inference/code/Users/dnn200001/output_frames/my_video_

In [11]:
%cd ~/cloudfiles/code/Users/dnn200001/

DETECT_ROOT = "yolov5/runs/detect"

classifier_interpreter = tf.lite.Interpreter(model_content=open("pose_classifier.tflite", "rb").read())
classifier_interpreter.allocate_tensors()
input_details = classifier_interpreter.get_input_details()
output_details = classifier_interpreter.get_output_details()

images_in_folder = DETECT_ROOT
images_out_folder = 'IGNORE'
csvs_out_path = 'poses_output.csv'



preprocessor = MoveNetPreprocessor(
      images_in_folder=images_in_folder,
      images_out_folder=images_out_folder,
      csvs_out_path=csvs_out_path,
  )

validCount = preprocessor.process(per_pose_class_limit=None)
#load the poses into dataframe for detection

poseToClassify, y, classes, df_test = load_pose_landmarks(csvs_out_path)

poseResults = poseInference(classifier_interpreter, poseToClassify)



/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-inference/code/Users/dnn200001
Skipped yolov5/runs/detect/MISS/frame_000156.jpg. No pose was confidentlly detected.
Skipped yolov5/runs/detect/MISS/frame_002704.jpg. No pose was confidentlly detected.
Skipped yolov5/runs/detect/MISS/frame_002852.jpg. No pose was confidentlly detected.
Skipped yolov5/runs/detect/MISS/frame_002853.jpg. No pose was confidentlly detected.
Skipped yolov5/runs/detect/MISS/frame_002856.jpg. No pose was confidentlly detected.
Skipped yolov5/runs/detect/MISS/frame_00300.jpg. No pose was confidentlly detected.
Skipped yolov5/runs/detect/MISS/frame_00315.jpg. No pose was confidentlly detected.
Skipped yolov5/runs/detect/MISS/frame_00345.jpg. No pose was confidentlly detected.
Skipped yolov5/runs/detect/MISS/frame_00375.jpg. No pose was confidentlly detected.
Skipped yolov5/runs/detect/MISS/frame_004052.jpg. No pose was confidentlly detected.
Skipped yolov5/runs/detect/MISS/frame_004202.jpg. No pose was confident

Preprocessing MISS
  0%|          | 0/98 [00:00<?, ?it/s]2023-04-27 18:32:41.278508: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 18:32:42.612154: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10601 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0001:00:00.0, compute capability: 3.7
/tmp/ipykernel_30923/485740632.py:130: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy

In [12]:
className = fileName

category2num = {
    "MISS": 0,
    "GOAL": 1,
    "ASSIST": 2,
    "STEAL": 3,
    "BLOCK": 4,
    "TURNOVER": 5,
    "REBOUND": 6
}

num2category = [category2num[key] for key in category2num.keys()]



import collections
fileName, counts = np.unique(classes.to_numpy(), return_counts=True)

output = []
start = 0

for num in counts:
    end = start + num
    output.append(poseResults[start:end])
    start = end

rows = 10
posesArr = [[] for i in range(rows)]

for poseCountI in range(len(output)):
    counter = collections.Counter(output[poseCountI])
    for i in range(10):
        #It return a percentage of each action
        posesArr[i].append(counter[i]/len(output[poseCountI]))


df = pd.DataFrame({
    'ball_in_hand': posesArr[0],
    'block': posesArr[1],
    'defense': posesArr[2],
    'dribble': posesArr[3],
    'no_action': posesArr[4],
    'pass': posesArr[5],
    'pick': posesArr[6],
    'run': posesArr[7],
    'shoot': posesArr[8],
    'walk': posesArr[9],
    'class': category2num[className]
})



   ball_in_hand     block   defense   dribble  no_action  pass  pick  \
0      0.023256  0.000000  0.000000  0.058140   0.093023   0.0   0.0   
1      0.048193  0.024096  0.000000  0.024096   0.204819   0.0   0.0   
2      0.048193  0.024096  0.000000  0.024096   0.204819   0.0   0.0   
3      0.022727  0.022727  0.005682  0.045455   0.193182   0.0   0.0   
4      0.030769  0.023077  0.023077  0.069231   0.180769   0.0   0.0   

        run     shoot      walk  class  
0  0.104651  0.011628  0.709302      0  
1  0.012048  0.036145  0.650602      0  
2  0.012048  0.036145  0.650602      0  
3  0.102273  0.005682  0.602273      0  
4  0.057692  0.015385  0.600000      0  


In [ ]:
X = df.drop(columns=['class'])
y = df['class']

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.9)

clf = RandomForestClassifier()
clf.fit(X_train,y_train)

predictions = clf.predict(X_test)

score = accuracy_score(y_test,predictions)
score